# Data Cleaning and Combining

In this file, data from various sources will be retrieved, combined and cleaned to produce a data set suitable for analysis and model building.

## Retrieving and Analyzing Data

In [132]:
# Import data science libraries
import pandas as pd
import numpy as np

# Import regex library
import re

# Import BeautifulSoup for HTML parsing and handling
from bs4 import BeautifulSoup

# Import NLTK for natural language processing
import nltk

In [27]:
# Configure Pandas to show all columns / rows
pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000

The first data set that will be examined for cleaning and use is the fraud_email_ data set.

In [133]:
# Read data from fraud_email_ data set and look at shape and dataframe info
fraud_df = pd.read_csv('./data/fraud_email_.csv')
print(fraud_df.shape)
print(fraud_df.info())

(11929, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11929 entries, 0 to 11928
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    11928 non-null  object
 1   Class   11929 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 186.5+ KB
None


In [134]:
# Look at content of data
fraud_df.head()

,Text,Class
0,Supply Quality China's EXCLUSIVE dimensions at...,1
1,over. SidLet me know. Thx.,0
2,"Dear Friend,Greetings to you.I wish to accost ...",1
3,MR. CHEUNG PUIHANG SENG BANK LTD.DES VOEUX RD....,1
4,Not a surprising assessment from Embassy.,0


In [35]:
# Look at split between fraud and regular emails
fraud_df['Class'].value_counts()

0    6742
1    5187
Name: Class, dtype: int64

Initial analysis of the fraud email data set reveals there is a sizable 11,929 rows of email text data with a classifier indicating whether the email the text was taken from was fraudulent or not. Although missing the email subject and from address is not ideal, this is a significant source of data which should be used in the data set.

The next data set that will be examined for use is the phishing_data_by_type data set.

In [135]:
# Pull the data set into a CSV and look at the shape and column info for the data frame
fraud_2_df = pd.read_csv('./data/phishing_data_by_type.csv')
print(fraud_2_df.shape)
print(fraud_2_df.info())

(159, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  157 non-null    object
 1   Text     159 non-null    object
 2   Type     159 non-null    object
dtypes: object(3)
memory usage: 3.9+ KB
None


In [136]:
# Look at first few entries of data set
fraud_2_df.head()

,Subject,Text,Type
0,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\...,Fraud
1,URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ...",Fraud
2,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,Fraud
3,from Mrs.Johnson,Goodday Dear\n\n\nI know this mail will come t...,Fraud
4,Co-Operation,FROM MR. GODWIN AKWESI\nTEL: +233 208216645\nF...,Fraud


In [137]:
# Look at the split for the content type
fraud_2_df['Type'].value_counts(normalize=True)

Fraud               0.251572
Phishing            0.251572
Commercial Spam     0.251572
False Positives     0.245283
Name: Type, dtype: float64

This data set contains only 159 documents but appears to contain a decent diversity of both phishing and fraud emails. It isn't ideal that there are also commercial spam emails since this may create an additional challenge for the identifier. As such this data set may have to undergo additional cleaning but is likely worth including.

In [39]:
phishing_extract_array = []
for i in range(1, 5):
    phishing_extract = pd.read_csv(f'./data/phishing_extract_{i}.csv')
    phishing_extract_array.append(phishing_extract)
    
combined_phishing_extract_df = pd.DataFrame()    
for df in phishing_extract_array:
    df.drop(columns=['from', 'subject'], inplace=True)
    combined_phishing_extract_df = pd.concat([combined_phishing_extract_df, df])
combined_phishing_extract_df.drop_duplicates(inplace=True)
combined_phishing_extract_df.to_csv('./data/phishing_eml_extract_full.csv', index=False)
#     print('---Head---')
#     display(df.head())
#     print('---Tail---')
#     display(df.tail())

In [40]:
combined_phishing_extract_df.duplicated().sum()

0

In [41]:
phishing_extract_df = pd.read_csv('./data/phishing_eml_extract_full.csv')
print(phishing_extract_df.shape)
display(phishing_extract_df)

(1871, 1)


,content
0,"Dear valued PayPal member, Due to recent fraud..."
1,Credit Union is constantly working to ensure s...
2,"Untitled Document Dear eBay Member, We regret ..."
3,Credit Union is constantly working to ensure s...
4,"Dear Amazon member, Dear member, Due to concer..."
5,"Dear eBay Member, We at eBay are sorry to info..."
6,We recently noticed one or more attempts to lo...
7,NCUA National Credit Union Administration Dear...
8,"Dear eBay member, Dear eBay User, During our r..."
9,"Security Center 128-bit encryptionAt PayPal, w..."


In [42]:
ham_extract_array = []
for i in range(1, 4):
    ham_extract = pd.read_csv(f'./data/ham_extract_{i}.csv')
    ham_extract_array.append(ham_extract)

combined_ham_extract_df = pd.DataFrame()    
for df in ham_extract_array:
    df.drop(columns=['from', 'subject'], inplace=True)
    combined_ham_extract_df = pd.concat([combined_ham_extract_df, df])
combined_ham_extract_df.drop_duplicates(inplace=True)
combined_ham_extract_df.to_csv('./data/ham_eml_extract_full.csv', index=False)

In [43]:
combined_ham_extract_df = pd.read_csv('./data/ham_eml_extract_full.csv')
print(combined_ham_extract_df.shape)
display(combined_ham_extract_df)

(1443, 1)


,content
0,I actually thought of this kind of active chat...
1,Content-Disposition: inline To view this newsl...
2,----- Original Message ----- From: Joseph S. B...
3,charsetISO-8859-1 formatflowed Bob We are a co...
4,I am delurking to comment on the Salon article...
5,Hi - I upgraded to 2.40 now 2.41 last week and...
6,"Hi all, I am wondering if anybody has successf..."
7,I installed Spamassassin 2.41 with Razor V2 th...
8,charsetWindows-1252 Im using Simple DNS from J...
9,"On Wed, Aug 14, 2002 at 10:59:22AM -0400, Thom..."


In [44]:
enron_email_df = pd.read_csv('./data/enron_extracted.csv')

## Data Frame Cleaning

In [52]:
phishing_2_clean_df = pd.DataFrame()

In [53]:
phishing_2_clean_df['content'] = fraud_df['Text']
phishing_2_clean_df['phishing'] = np.where(
    fraud_df['Class'] == 1,
    1,
    0
)

In [54]:
phishing_2_clean_df.head()

,content,phishing
0,Supply Quality China's EXCLUSIVE dimensions at...,1
1,over. SidLet me know. Thx.,0
2,"Dear Friend,Greetings to you.I wish to accost ...",1
3,MR. CHEUNG PUIHANG SENG BANK LTD.DES VOEUX RD....,1
4,Not a surprising assessment from Embassy.,0


In [55]:
fraud_2_df.head()

,Subject,Text,Type
0,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\...,Fraud
1,URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ...",Fraud
2,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,Fraud
3,from Mrs.Johnson,Goodday Dear\n\n\nI know this mail will come t...,Fraud
4,Co-Operation,FROM MR. GODWIN AKWESI\nTEL: +233 208216645\nF...,Fraud


In [56]:
phishing_3_clean_df = pd.DataFrame()
phishing_3_clean_df['content'] = fraud_2_df['Text']
phishing_3_clean_df['phishing'] = np.where(
    (fraud_2_df['Type'] == 'Fraud') | (fraud_2_df['Type'] == 'Phishing'),
    1,
    0
)
phishing_3_clean_df = phishing_3_clean_df[fraud_2_df['Type'] != 'Commercial Spam']

In [57]:
phishing_3_clean_df.head()

,content,phishing
0,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\...,1
1,"Dear Friend,\n\nI am Mr. Ben Suleman a custom ...",1
2,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,1
3,Goodday Dear\n\n\nI know this mail will come t...,1
4,FROM MR. GODWIN AKWESI\nTEL: +233 208216645\nF...,1


In [58]:
phishing_5_clean_df = phishing_extract_df
phishing_5_clean_df['phishing'] = 1

In [59]:
phishing_6_clean_df = combined_ham_extract_df
phishing_6_clean_df['phishing'] = 0

In [60]:
phishing_all_df = pd.concat([
    phishing_2_clean_df,
    phishing_3_clean_df,
    phishing_5_clean_df,
    phishing_6_clean_df
], ignore_index=True)

In [61]:
phishing_all_df.dropna(inplace=True)

In [62]:
phishing_all_df

,content,phishing
0,Supply Quality China's EXCLUSIVE dimensions at...,1
1,over. SidLet me know. Thx.,0
2,"Dear Friend,Greetings to you.I wish to accost ...",1
3,MR. CHEUNG PUIHANG SENG BANK LTD.DES VOEUX RD....,1
4,Not a surprising assessment from Embassy.,0
...,...,...
15357,You appear to be using an email application th...,0
15358,CNET Investor Dispatch Quote LookupEnter symbo...,0
15359,Todays Headlines from The Register -----------...,0
15360,"Hi Everyone, There seem to be several bonehead...",0


In [63]:
def extractHTMLText(html):
    soup = BeautifulSoup(html, features="html.parser")
    text = soup.get_text()
    return text

In [64]:
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rosswillett/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/rosswillett/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [65]:
nltk_eng_words = set(nltk.corpus.words.words())
def extractTextContent (text):
    filteredText = text.replace('=2C', '')
    characterFilteredText = re.sub(r'[^a-zA-Z\s]', ' ', filteredText)
    englishWordOnlyTextArr = []
    for word in nltk.word_tokenize(characterFilteredText):
        lower_word = word.lower()
        if lower_word in nltk_eng_words and len(word) > 1:
            englishWordOnlyTextArr.append(word)
        elif lower_word == 'i' or lower_word == 'a':
            englishWordOnlyTextArr.append(word)
    return ' '.join(englishWordOnlyTextArr)

In [66]:
def getWordCount(text):
    # allow alphanumeric characters and spaces only
    alpha_only_text = re.sub(r'[^a-zA-Z\s]', '', text)
    no_space_alpha_text = re.sub(r'\s\s+', ' ', text)
    
    # split sentence into words
    listofwords = alpha_only_text.split(' ')
    
    return len(listofwords)

In [67]:
rows_with_html = phishing_all_df['content'].str.lower().str.contains('</', na=True)

In [68]:
phishing_all_df.loc[rows_with_html, ['content']] = phishing_all_df[rows_with_html]['content'].apply(extractHTMLText)

In [69]:
phishing_all_df['unsecure_link_count'] = phishing_all_df['content'].str.count('http://')

In [70]:
phishing_all_df['secure_link_count'] = phishing_all_df['content'].str.count('https://')

In [71]:
phishing_all_df['numbers_count'] = phishing_all_df['content'].apply(lambda text: len(re.findall(r'\d+', text)))

In [72]:
phishing_all_df.loc[:, ['content']] = phishing_all_df['content'].apply(extractTextContent)

In [73]:
phishing_all_df['word_count'] = phishing_all_df['content'].apply(getWordCount)

In [74]:
phishing_all_df = phishing_all_df[phishing_all_df['content'] != '']

In [75]:
phishing_all_df.isna().sum()

content                0
phishing               0
unsecure_link_count    0
secure_link_count      0
numbers_count          0
word_count             0
dtype: int64

In [76]:
phishing_all_df.duplicated().sum()

1680

In [77]:
phishing_all_df.drop_duplicates(inplace=True, ignore_index=True)

/var/folders/t9/4whyp65x6cqg_kdhc89_f_080000gn/T/ipykernel_6688/2433033315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phishing_all_df.drop_duplicates(inplace=True, ignore_index=True)


In [78]:
phishing_all_df

,content,phishing,unsecure_link_count,secure_link_count,numbers_count,word_count
0,Supply Quality China EXCLUSIVE at Unbeatable P...,1,0,0,10,131
1,over me know,0,0,0,0,3
2,Dear Friend to you I wish to accost you with a...,1,0,0,9,385
3,BANK BRANCH CENTRAL HONG HONG Let me start by ...,1,1,0,6,549
4,Not a surprising assessment from Embassy,0,0,0,0,6
...,...,...,...,...,...,...
12930,You appear to be an application that wont prop...,0,2,0,40,1041
12931,Investor Dispatch Quote symbol Symbol My Portf...,0,0,0,136,496
12932,from The Register To from this daily news upda...,0,24,0,60,298
12933,Hi Everyone There seem to be several in our cl...,0,2,0,2,91


In [136]:
phishing_all_df.to_csv('./data/phishing_all_data_clean_3.csv', index=False)

### Add Enron

In [94]:
phishing_2_fraud_only = phishing_2_clean_df[phishing_2_clean_df['phishing'] == 1]

In [45]:
enron_internal_df = enron_email_df[enron_email_df['from'].str.contains('@enron')]

In [46]:
enron_internal_df.sample(10)

,from,to,subject,content
52493,shirley.crenshaw@enron.com,move-team@enron.com,Computers from Research Group,Good morning all: This past weekend you moved ...
20193,kate.symes@enron.com,andy.chen@enron.com,Re: Warning,Thanks - Enron designed that stationary specif...
447804,darrell.schoolcraft@enron.com,"steve.january@enron.com, kimberly.watson@enron...",TW Weekend scheduled volumes,March 2002 Scheduled Scheduled Friday 15 West ...
89831,ted.murphy@enron.com,"s..bradford@enron.com, r..brackett@enron.com, ...",Quick Update,"Bill and Friends: FYI, things are still quite ..."
250452,lynn.blair@enron.com,"tim.johanson@enron.com, john.williams@enron.co...",RE: Requested training by Xcel in Denver,"Tim and Randy, how did the traning go Thanks. ..."
62105,rebecca.mcdonald@enron.com,"jeff.skilling@enron.com, kevin.hannon@enron.com",FW: Sale of Enron's Interest in Bachaquero,FYI -----Original Message----- From: Tortolero...
486224,maria.sandoval@enron.com,"asandov225@aol.com, andrea.guillen@enron.com, ...",The Empty Chair,THE EMPTY CHAIR A mans daughter had asked the ...
109654,michele.winckowski@enron.com,NaN,FW: Something Worth Seeing,This is very powerful. Some photos are very gr...
308442,frank.davis@enron.com,tana.jones@enron.com,Pulp & Paper Long Descriptions,"Tana, Attached below are examples of EnronOnli..."
457221,maureen.mcvicker@enron.com,tom.briggs@enron.com,Re: Draft Wyden letter,Tom: What address should I use for the Sen. Wy...


In [115]:
enron_internal_clean = pd.DataFrame()
enron_internal_clean = enron_internal_df.copy()
enron_internal_clean.loc[:, 'phishing'] = 0
enron_internal_clean.drop(columns=['to', 'from', 'subject'], inplace=True)
enron_internal_clean_sample = enron_internal_clean.sample(5656)

In [116]:
new_df = pd.concat([
    phishing_2_fraud_only,
    phishing_3_clean_df,
    phishing_5_clean_df,
    phishing_6_clean_df,
    enron_internal_clean_sample
], ignore_index=True)

In [117]:
new_df['phishing'].value_counts()

1    7138
0    7138
Name: phishing, dtype: int64

In [118]:
new_df.dropna(inplace=True)

In [119]:
rows_with_html = new_df['content'].str.lower().str.contains('</', na=True)

In [120]:
new_df.loc[rows_with_html, ['content']] = new_df[rows_with_html]['content'].apply(extractHTMLText)

In [121]:
new_df['unsecure_link_count'] = new_df['content'].str.count('http://')

In [122]:
new_df['secure_link_count'] = new_df['content'].str.count('https://')

In [123]:
new_df['numbers_count'] = new_df['content'].apply(lambda text: len(re.findall(r'\d+', text)))

In [124]:
new_df.loc[:, ['content']] = new_df['content'].apply(extractTextContent)

In [125]:
new_df['word_count'] = new_df['content'].apply(getWordCount)

In [126]:
new_df = new_df[new_df['content'] != '']

In [127]:
new_df.isna().sum()

content                0
phishing               0
unsecure_link_count    0
secure_link_count      0
numbers_count          0
word_count             0
dtype: int64

In [128]:
new_df.duplicated().sum()

1474

In [129]:
new_df.drop_duplicates(inplace=True, ignore_index=True)

In [130]:
new_df

,content,phishing,unsecure_link_count,secure_link_count,numbers_count,word_count
0,Supply Quality China EXCLUSIVE at Unbeatable P...,1,0,0,10,131
1,Dear Friend to you I wish to accost you with a...,1,0,0,9,385
2,BANK BRANCH CENTRAL HONG HONG Let me start by ...,1,1,0,6,549
3,from barrister friend I know that my letter wi...,1,0,0,41,527
4,SOLICITING FOR A BUSINESS VENTURE AND DEAR SIR...,1,0,0,20,323
...,...,...,...,...,...,...
12746,You know I must have received your message on ...,0,0,0,5,82
12747,AGRICULTURE Soft commodity find the going hard...,0,3,0,358,7277
12748,I have your resume with my commentary to Bibi ...,0,0,0,0,31
12749,Mark How should we handle this In the past I h...,0,0,0,50,274


In [131]:
new_df.to_csv('./data/fraud_with_enron_data_clean_1.csv', index=False)